In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pandas as pd

In [38]:
driver = webdriver.Chrome()

# URL of MagicBricks PG hostels in Hyderabad
url = "https://www.magicbricks.com/property-for-rent/residential-paying-guest?city=2060"

driver.get(url)
time.sleep(15)  # Wait for the initial load

pg_hostels = []
scroll_pause_time = 15 # Pause to wait for loading new data
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load new content
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
    # Get page source and parse with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract data
    listings = soup.find_all('div', class_='m-srp-card clearfix newLaunched')  # Update class name based on the actual HTML structure
    for listing in listings:
        try:
            title = listing.find('span', class_='m-srp-card__title__name').text.strip()
        except:
            title = None
    
        try:
            location = listing.find('div', class_='m-srp-card__link m-srp-card__link--nearby').text.strip()
        except:
            location = None
    
        try:
            price = listing.find('div', class_='m-srp-card__price').text.strip()
        except:
            price = None

        try:
            room_info = listing.find('div', class_='m-srp-card__roomInfo__room-detail clearfix').text.strip()
        except:
            price = None

        try:
            type = listing.find('div', class_='m-srp-card__info flex__item').text.strip()
        except:
            price = None

        try:
            places = listing.find('div', class_='m-srp-card__tuple__flex').text.strip()
        except:
            places = None
    
        pg_hostels.append({
            'Title': title,
            'Location': location,
            'Price': price,
            'Room Info':room_info,
            'Type':type,
            'Places':places
        })

# Convert to DataFrame and save to CSV
df = pd.DataFrame(pg_hostels)

# Close the WebDriver
driver.quit()

print("Scraping complete.")
df

Scraping complete.


,Title,Location,Price,Room Info,Type,Places
0,Zolo Berry PG/Paying Guest,in Madhapur,"₹13,600 Onwards","Twin Sharing With AC ₹13,600 Single Room With...","Coed₹13,600 OnwardsFood Charge extraBeds Avail...",0.7 Km from Ravindra Bharthi School 1.4 Km fro...
1,Zolo Aastha PG/Paying Guest,in Manikonda Jagir,"₹6,196 Onwards","Single Room With AC ₹19,845 Twin Sharing With...","Coed₹6,196 OnwardsBeds Available Zolo Aastha P...",0.1 Km from Tejomayi Schools 0.5 Km from Manik...
2,Aspire Comforts Womens PG/Paying Guest,"in Kukatpally Housing Board Colony, NH 9","₹6,000 Onwards","Single Room ₹12,000 Twin Sharing ₹9,000 Trip...","Girls₹6,000 OnwardsFood IncludedBeds Available...",0.0 Km from Vali Rameshwars IAS Academy 0.1 Km...
3,LAXMI VENKTESWARA CO-LIVING PG/Paying Guest AN...,in Kothaguda,"₹8,500 Onwards","Single Room With AC ₹26,000 Twin Sharing With...","Coed₹8,500 OnwardsFood IncludedBeds Available ...",0.2 Km from Sri Gautham Grammar High School 0....
4,SV VIHANG CO LIVING PG/Paying Guest,in TNGOS Colony,"₹7,500 Onwards","Single Room ₹16,000 Twin Sharing ₹8,000 Trip...","Coed₹7,500 OnwardsBeds Available SV VIHANG CO ...",0.7 Km from Delhi World School Gachibowli 2.1 ...
...,...,...,...,...,...,...
877,Ashok Reddy PG/Paying Guest for Men,in Kondapur,"₹6,500 Onwards","Single Room ₹18,000 Twin Sharing ₹9,000 Trip...","Boys₹6,500 OnwardsFood IncludedBeds Available ...",0.0 Km from Chirec International School 1.2 Km...
878,Nakshatra Castle Coliving and Guest INN PG/Pay...,in Hitech City,"₹10,000 Onwards","Single Room ₹25,000 Twin Sharing ₹13,000 Tri...","Coed₹10,000 OnwardsFood IncludedBeds Available...",0.9 Km from Sri Venkateshwara Group Of College...
879,BSR Executive Mens PG/Paying Guest,in Gachibowli,"₹6,500 Onwards","Single Room With AC ₹18,000 Twin Sharing ₹9,0...","Boys₹6,500 OnwardsFood IncludedBeds Available ...",0.6 Km from Dr.K.S.Rao's IAS Academy 0.2 Km fr...
880,Sri Chakra Colive PG/Paying Guest,in Rai Durg,"₹9,500 Onwards","Single Room ₹28,000 Twin Sharing ₹13,000 Tri...","Coed₹9,500 OnwardsFood IncludedBeds Available ...",0.0 Km from Petals 0.0 Km from Arabindo Orbit ...


In [44]:
df.to_csv("pghostels.csv")

In [45]:
df

,Title,Location,Price,Room Info,Type,Places
0,Zolo Berry PG/Paying Guest,in Madhapur,"₹13,600 Onwards","Twin Sharing With AC ₹13,600 Single Room With...","Coed₹13,600 OnwardsFood Charge extraBeds Avail...",0.7 Km from Ravindra Bharthi School 1.4 Km fro...
1,Zolo Aastha PG/Paying Guest,in Manikonda Jagir,"₹6,196 Onwards","Single Room With AC ₹19,845 Twin Sharing With...","Coed₹6,196 OnwardsBeds Available Zolo Aastha P...",0.1 Km from Tejomayi Schools 0.5 Km from Manik...
2,Aspire Comforts Womens PG/Paying Guest,"in Kukatpally Housing Board Colony, NH 9","₹6,000 Onwards","Single Room ₹12,000 Twin Sharing ₹9,000 Trip...","Girls₹6,000 OnwardsFood IncludedBeds Available...",0.0 Km from Vali Rameshwars IAS Academy 0.1 Km...
3,LAXMI VENKTESWARA CO-LIVING PG/Paying Guest AN...,in Kothaguda,"₹8,500 Onwards","Single Room With AC ₹26,000 Twin Sharing With...","Coed₹8,500 OnwardsFood IncludedBeds Available ...",0.2 Km from Sri Gautham Grammar High School 0....
4,SV VIHANG CO LIVING PG/Paying Guest,in TNGOS Colony,"₹7,500 Onwards","Single Room ₹16,000 Twin Sharing ₹8,000 Trip...","Coed₹7,500 OnwardsBeds Available SV VIHANG CO ...",0.7 Km from Delhi World School Gachibowli 2.1 ...
...,...,...,...,...,...,...
877,Ashok Reddy PG/Paying Guest for Men,in Kondapur,"₹6,500 Onwards","Single Room ₹18,000 Twin Sharing ₹9,000 Trip...","Boys₹6,500 OnwardsFood IncludedBeds Available ...",0.0 Km from Chirec International School 1.2 Km...
878,Nakshatra Castle Coliving and Guest INN PG/Pay...,in Hitech City,"₹10,000 Onwards","Single Room ₹25,000 Twin Sharing ₹13,000 Tri...","Coed₹10,000 OnwardsFood IncludedBeds Available...",0.9 Km from Sri Venkateshwara Group Of College...
879,BSR Executive Mens PG/Paying Guest,in Gachibowli,"₹6,500 Onwards","Single Room With AC ₹18,000 Twin Sharing ₹9,0...","Boys₹6,500 OnwardsFood IncludedBeds Available ...",0.6 Km from Dr.K.S.Rao's IAS Academy 0.2 Km fr...
880,Sri Chakra Colive PG/Paying Guest,in Rai Durg,"₹9,500 Onwards","Single Room ₹28,000 Twin Sharing ₹13,000 Tri...","Coed₹9,500 OnwardsFood IncludedBeds Available ...",0.0 Km from Petals 0.0 Km from Arabindo Orbit ...


In [3]:
from IPython.display import FileLink

# Show a clickable download link in notebook
FileLink("pghostels.csv")

C:\Users\Dell\pghostels.csv